In [1]:
import numpy as np

import torch
import torch.optim as optim
import torch.nn as nn
from FairRanking.models.DirectRanker import DirectRanker
from FairRanking.helpers import nDCG_cls, disparate_impact


In [2]:
from FairRanking.datasets.adult import Adult
from FairRanking.datasets.law import Law
from FairRanking.datasets.compas import Compas
from FairRanking.datasets.wiki import Wiki

#data = Law('race')
data = Adult()
#data = Compas()
#data = Wiki()
(X_train, s_train, y_train), (X_val, s_val, y_val), (X_test, s_test, y_test) = data.get_data()

In [3]:
# has to go to the helpers function
def calc_accuracy(outputs, labels):
    pred = (outputs > 0).int()
    correct_predictions = (pred == labels).int()
    accuracy = correct_predictions.sum() / len(labels)
    return accuracy

In [4]:
from FairRanking.models.BaseDirectRanker import build_pairs
# Needs to got to BaseDatasets
def convert_data_to_tensors(data):
    (X_train, s_train, y_train), (X_val, s_val, y_val), (X_test, s_test, y_test) = data.get_data()
    X_train0, X_train1, s_train0, s_train1, y_train = build_pairs(X_train, y_train, s_train, X_train.shape[0])
    X_val0, X_val1, s_val0, s_val1, y_val = build_pairs(X_val, y_val, s_val, X_val.shape[0])
    X_test0, X_test1, s_test0, s_test1, y_test = build_pairs(X_test, y_test, s_test, X_test.shape[0])
    return X_train0, X_train1, s_train0, s_train1, y_train, X_val0, X_val1, s_val0, s_val1, y_val, X_test0, X_test1, s_test0, s_test1, y_test




In [5]:
X_train0, X_train1, s_train0, s_train1, y_train, X_val0, X_val1, s_val0, s_val1, y_val, X_test0, X_test1, s_test0, s_test1, y_test = convert_data_to_tensors(data)

/Users/robert/Desktop/Bachelor/FairRanker/FairRanking/models/BaseDirectRanker.py:231: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /private/var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_9d63z49rj_/croot/pytorch_1681837279022/work/torch/csrc/utils/tensor_new.cpp:233.)
  x0 = torch.tensor(x0, dtype=torch.float32)


In [6]:
X_train0.shape, y_train.shape

(torch.Size([20838, 55]), torch.Size([20838, 1]))

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import os

def create_writer(dataset: str,
                  experiment_name: str,
                  model_name: str,
                  extra: str = None):
    
    timestamp = datetime.now().strftime('%Y-%m-%d')

    if extra:
        log_dir = os.path.join("runs", dataset, timestamp, experiment_name, model_name, extra)
    else:
        log_dir = os.path.join("runs", dataset, timestamp, experiment_name, model_name, extra)
    print(f"[INFO] Created SummaryWriter saving to {log_dir}")
    return SummaryWriter(log_dir=log_dir)

In [6]:
def calc_accuracy(outputs, labels):
    pred = (outputs > 0).int()
    correct_predictions = (pred == labels).int()
    accuracy = correct_predictions.sum() / len(labels)
    return accuracy

In [ ]:
y_val.shape, y_train.shape, y_test.shape

In [ ]:
writer.close()

In [8]:
# law dataset works
# adult have 0 gradients after a few epoch, regardless model complexity
# compas only learns very slow, regardless model complexity
from datetime import datetime
y_train_same = torch.zeros_like(y_train) # for same relevance classes
hidden_layers = [64, 32, 16]
model = DirectRanker(num_features=X_train0.shape[1],
                     kernel_initializer=nn.init.normal_,
                     hidden_layers=hidden_layers) # more hidden layers => gradient becomes zero and loss high
n_epochs = 1000
lr = 0.1
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.MSELoss(reduction='mean') 
#loss_fn = nn.BCEWithLogitsLoss(reduction='mean')
#y_train = torch.squeeze(y_train)
# training
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(X_train0, X_train1)
    #y_pred = torch.squeeze(y_pred)
    loss = loss_fn(y_train, y_pred)
    train_disparate_impact = disparate_impact(y_pred, s_train0, s_train1)
    loss.backward()
    optimizer.step()
    # same relevance classes
    optimizer.zero_grad()
    #dim = 0
    #idx = torch.randperm(X_train1.shape[dim])
    #t_shuffled = X_train1[idx]
    #y_pred_same = model(X_train1, t_shuffled)
    #loss_same = loss_fn(y_train_same, y_pred_same)
    #loss_same.backward()
    #optimizer.step()
    model.eval()
    y_val_pred = model(X_val0, X_val1)
    #y_val_pred = torch.squeeze(y_val_pred)
    val_loss = loss_fn(y_val, y_val_pred)
    train_loss = loss.item()
    val_loss = val_loss.item()
    val_disparate_impact = disparate_impact(y_val_pred, s_val0, s_val1)
    train_acc = calc_accuracy(y_pred, y_train)
    val_acc = calc_accuracy(y_val_pred, y_val)
    if epoch % 40 == 0:   
        print(f'Epoch: {epoch}/{n_epochs}\t Training Loss: {train_loss:.2f}  Training Accuracy: {train_acc:.2f}  DI: {train_disparate_impact:.2f}' \
              f'\t Validation Loss: {val_loss:.2f}  Validation Accuracy: {val_acc:.2f}  DI: {val_disparate_impact:.2f}')
        #print(f'Same classes Loss: {loss_same:.2f}\t Same classes Acc: {calc_accuracy(y_pred_same, y_train_same):.2f}')

model.eval()
y_test_pred = model(X_test0, X_test1)
test_loss = loss_fn(y_test, y_test_pred)
test_acc = calc_accuracy(y_test_pred, y_test)
test_disparate_impact = disparate_impact(y_test_pred, s_test0, s_test1)
print(f'Test Loss: {test_loss.item():.2f}\t Test Accuracy: {test_acc.item():.2f}\t DI: {test_disparate_impact:.2f}')
print('finished')

Epoch: 0/1000	 Training Loss: 0.98  Training Accuracy: 0.63  DI: 9.21	 Validation Loss: 0.56  Validation Accuracy: 0.80  DI: 13.78
Epoch: 40/1000	 Training Loss: 0.27  Training Accuracy: 0.91  DI: 15.84	 Validation Loss: 0.34  Validation Accuracy: 0.89  DI: 14.10
Epoch: 80/1000	 Training Loss: 0.21  Training Accuracy: 0.92  DI: 15.93	 Validation Loss: 0.34  Validation Accuracy: 0.88  DI: 14.04
Epoch: 120/1000	 Training Loss: 0.20  Training Accuracy: 0.93  DI: 16.06	 Validation Loss: 0.38  Validation Accuracy: 0.88  DI: 14.45
Epoch: 160/1000	 Training Loss: 0.15  Training Accuracy: 0.94  DI: 15.97	 Validation Loss: 0.42  Validation Accuracy: 0.87  DI: 14.00
Epoch: 200/1000	 Training Loss: 0.13  Training Accuracy: 0.95  DI: 15.99	 Validation Loss: 0.41  Validation Accuracy: 0.88  DI: 13.82
Epoch: 240/1000	 Training Loss: 0.12  Training Accuracy: 0.96  DI: 16.09	 Validation Loss: 0.41  Validation Accuracy: 0.88  DI: 13.88
Epoch: 280/1000	 Training Loss: 0.11  Training Accuracy: 0.96  DI: 

In [ ]:
y_test_pred = model(X_test1, X_test0)
test_loss = loss_fn(y_test, y_test_pred)
test_acc = calc_accuracy(y_test_pred, y_test)
print(f'Test Loss: {test_loss.item():.2f}\t Test Accuracy: {test_acc.item():.2f}')

In [ ]:
# Testing Antisymmetry
model.eval()
first_output = model(X_test0, X_test1)
second_output = model(X_test1, X_test0)
antisymmetry_test = torch.all(first_output == (-1)*second_output)
print(antisymmetry_test)

In [ ]:
# Testing Reflexivity
model.eval()
reflexivity_test = model(X_test0, X_test0)
print(torch.count_nonzero(reflexivity_test)) # should be zero
reflexivity_test = model(X_test1, X_test1)
print(torch.count_nonzero(reflexivity_test)) # should be zero

In [ ]:
# same relevance class but different
model.eval()
output1 = model(X_train1[2], X_train1[4])
output2 = model(X_test0[4], X_test0[1])
output1, output2

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir runs/

In [ ]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

In [14]:
hidden_layers = [64, 32, 16]
model = DirectRanker(num_features=X_train0.shape[1], hidden_layers=hidden_layers)
model.eval()
y_pred = model(X_train1, X_train0)
train_ndcg = nDCG_cls(y_pred, y_train, at=float('inf'))

print(y_pred)
print(train_ndcg)

[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0]

# Training model without a writer

In [9]:
# law dataset works
# adult have 0 gradients after a few epoch, regardless model complexity
# compas only learns very slow, regardless model complexity
hidden_layers = [64, 32, 16]
model = DirectRanker(num_features=X_train0.shape[1],
                     kernel_initializer=nn.init.xavier_uniform_,
                     hidden_layers=hidden_layers) # more hidden layers => gradient becomes zero and loss high
n_epochs = 1000
lr = 0.01
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.MSELoss(reduction='mean') 
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    y_pred = model(X_train1, X_train0)
    loss = loss_fn(y_train, y_pred)
    loss.backward()
    optimizer.step()
    # same relevance classes
    model.eval()
    y_val_pred = model(X_val1, X_val0)
    val_loss = loss_fn(y_val, y_val_pred)
    train_loss = loss.item()
    val_loss = val_loss.item()
    train_acc = calc_accuracy(y_pred, y_train)
    val_acc = calc_accuracy(y_val_pred, y_val)
    train_ndcg = nDCG_cls(y_pred, y_train)
    val_ndcg = nDCG_cls(y_val_pred, y_val)
    if epoch % 40 == 0:   
        print(f'Epoch: {epoch}/{n_epochs}\t Training Loss: {train_loss:.2f}  Training Accuracy: {train_acc:.2f}' \
              f'\t Validation Loss: {val_loss:.2f}  Validation Accuracy: {val_acc:.2f}' \
              f'\n\tTrain nDCG: {train_ndcg:.2f}  Validation nDCG: {val_ndcg:.2f}')
model.eval()
y_test_pred = model(X_test1, X_test0)
test_loss = loss_fn(y_test, y_test_pred)
test_acc = calc_accuracy(y_test_pred, y_test)
test_ndcg = nDCG_cls(y_test_pred, y_test)
print(f'Test Loss: {test_loss.item():.2f}\t Test Accuracy: {test_acc.item():.2f}\t Test nDCG: {train_ndcg:.2f}')
print('finished')

[[1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0], [1.0]

KeyboardInterrupt: 

In [ ]:
import torch
from torch import nn
from torch.autograd import Function

# This function will reverse the gradient
class GradientReversalFunction(Function):
    @staticmethod
    def forward(ctx, x):
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        # Reverse the gradient
        return -grad_output * gamma  # gamma is a hyperparameter to scale the reversed gradient

def grad_reverse(x, gamma=1.0):
    return GradientReversalFunction.apply(x)

# Assuming nn1, nn2, nn1bias, nn2bias are defined and part of YourModel class
# ...

# During the training loop
for data in dataloader:
    # Forward pass through nn1 and nn2
    features1 = nn1(document1)
    features2 = nn2(document2)

    # Forward pass through nn1bias and nn2bias
    sensitive_pred1 = nn1bias(features1)
    sensitive_pred2 = nn2bias(features2)

    # Compute the sensitive attribute prediction loss
    Lbias1 = sensitive_criterion(sensitive_pred1, sensitive_labels1)
    Lbias2 = sensitive_criterion(sensitive_pred2, sensitive_labels2)
    Lbias = Lbias1 + Lbias2  # Total sensitive loss

    # Backward pass of the sensitive loss with gradient reversal
    optimizer.zero_grad()  # Clear existing gradients
    reversed_features1 = grad_reverse(features1, gamma)
    reversed_features2 = grad_reverse(features2, gamma)
    Lbias.backward()  # Gradients are reversed during this backward pass

    # Now update nn1 and nn2 with the reversed gradients
    optimizer.step()

    # Compute the ranking loss (o1)
    optimizer.zero_grad()  # Clear gradients again
    ranking_loss = ranking_criterion(o1, true_ranking_labels)
    ranking_loss.backward()  # Normal backward pass, without gradient reversal

    # Update the entire model with gradients from the ranking task
    optimizer.step()


In [7]:
def calc_sens_loss(sensitive0, sensitive1, s_true, gamma=1.0):
    loss = 0
    loss += -s_true * torch.log2(sensitive0) - (1 - s_true) * torch.log2(1 - sensitive0)
    loss += -s_true * torch.log2(sensitive1) - (1 - s_true) * torch.log2(1 - sensitive1)
    return gamma * loss


In [8]:
from FairRanking.models.DirectRankerAdv import DirectRankerAdv
# law dataset works
# adult have 0 gradients after a few epoch, regardless model complexity
# compas only learns very slow, regardless model complexity
from datetime import datetime
y_train_same = torch.zeros_like(y_train) # for same relevance classes
hidden_layers = [64, 32, 16]
model = DirectRankerAdv(num_features=X_train0.shape[1],
                     kernel_initializer=nn.init.xavier_uniform_,
                     hidden_layers=hidden_layers)
n_epochs = 1000
lr = 0.01
optimizer = optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.MSELoss(reduction='mean') 
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    y_pred, sensitive_pred0, sensitive_pred1 = model(X_train1, X_train0)
    main_loss = loss_fn(y_train, y_pred)
    main_loss.backward(retain_graph=True)
    optimizer.step()
    optimizer.zero_grad()
    sensitive_loss = calc_sens_loss(sensitive_pred0, sensitive_pred1, s_train, gamma=1.0)
    sensitive_loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    y_val_pred, sensitive_val_pred0, sensitive_val_pred1 = model(X_val1, X_val0)
    val_loss = loss_fn(y_val, y_val_pred)
    sensitive_val_loss = calc_sens_loss(sensitive_val_pred0, sensitive_val_pred1, s_val, gamma=1.0)
    train_loss = loss.item()
    val_loss = val_loss.item()
    train_acc = calc_accuracy(y_pred, y_train)
    val_acc = calc_accuracy(y_val_pred, y_val)
    if epoch % 40 == 0:   
        print(f'Epoch: {epoch}/{n_epochs}\t Training Loss: {train_loss:.2f}  Training Accuracy: {train_acc:.2f}  Sensitive Loss: {sensitive_loss:.2f}' \
              f'\t Validation Loss: {val_loss:.2f}  Validation Accuracy: {val_acc:.2f}  Validation Sens: {sensitive_val_loss:.2f}')

model.eval()
y_test_pred = model(X_test1, X_test0)
test_loss = loss_fn(y_test, y_test_pred)
test_acc = calc_accuracy(y_test_pred, y_test)
print(f'Test Loss: {test_loss.item():.2f}\t Test Accuracy: {test_acc.item():.2f}')
writer.close()
print('finished')

TypeError: Module.__init__() takes 1 positional argument but 2 were given